In [ ]:
import json
import os

input_file = "raw_dataset/train.jsonl"
output_file = "processed/train_instruction.jsonl"

In [ ]:
instruction_data = []

with open(input_file, "r") as fin:
    for line in fin:
        entry = json.loads(line)
        print("🔍 RAW ENTRY SAMPLE:")
        print(json.dumps(entry, indent=2))
        break
        abstract = entry.get("abstract", "").strip()
        questions = entry.get("question", [])
        answers = entry.get("answers", [])

        for i in range(len(questions)):
            question = questions[i].strip()
            answer_block = answers[i]

            if not isinstance(answer_block, dict):
                continue

            annotations = answer_block.get("answer", [])
            if not annotations or not isinstance(annotations[0], dict):
                continue

            ann = annotations[0]

            if ann.get("unanswerable", False):
                continue

            # Prefer free_form_answer
            answer = ann.get("free_form_answer", "").strip()

            # Fallback to extractive_span
            if not answer:
                spans = ann.get("extractive_spans", [])
                if spans and isinstance(spans[0], str):
                    answer = spans[0].strip()

            if not answer:
                continue

            explanation = ann.get("highlighted_evidence", [])
            explanation_text = explanation[0].strip() if explanation else ""

            input_text = f"{abstract}\nQuestion: {question}"
            output_text = f"{answer}. Explanation: {explanation_text}" if explanation_text else answer

            instruction_data.append({
                "input": input_text,
                "output": output_text
            })


✅ Converted 0 QA pairs to processed/train_instruction.jsonl
